In [10]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

In [11]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [12]:
class CallOption:
    
    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

    def BSprice(self, dynamics):
        # ignores self.price if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S*np.exp(dynamics.rGrow*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d1 = np.log(F/self.K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-dynamics.r*self.T)*(F*norm.cdf(d1)-self.K*norm.cdf(d2))
    
    def trysigma(self, inputsigma, dynamics, targetprice):
        dynamics.sigma = inputsigma
        return self.BSprice(dynamics) - targetprice

    
    def IV(self, dynamics):
        # ignores dynamics.sigma, because this function solves for sigma.  
        
        if self.price is None: 
            raise ValueError('Contract price must be given')
    
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-self.K)*df])
        C = self.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan 

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
        hi = dytry.sigma
        lo = hi/2

        impliedVolatility = brentq(self.trysigma, hi, lo, args=(dynamics, self.price), maxiter=1000)
        return impliedVolatility

In [13]:
# first IV
dynamics = GBMdynamics(S=100, r=0.05, rGrow=0.05)
contract1 = CallOption(K=100, T=0.1, price=5.25)
imp_vol_1 = contract1.IV(dynamics)
imp_vol_1

0.397320385795576

In [14]:
# second IV
contract2 = CallOption(K=100, T=0.2, price=7.25)
imp_vol_2 = contract2.IV(dynamics)
imp_vol_2

0.380171291551054

In [15]:
# thrid IV
contract3 = CallOption(K=100, T=0.5, price=9.5)
imp_vol_3 = contract3.IV(dynamics)
imp_vol_3

0.2950972521756794

In [16]:
def bs_call_formula(X, t, K, T, rGrow, r, sigma):
    F = X*np.exp(rGrow*(T-t))
    d1 = np.log(F/K)/(sigma*np.sqrt(T-t)) + sigma*np.sqrt(T-t)/2
    d2 = np.log(F/K)/(sigma*np.sqrt(T-t)) - sigma*np.sqrt(T-t)/2
    call_price = np.exp(-r*(T-t))*(F*norm.cdf(d1) - K*norm.cdf(d2))
    return call_price

In [18]:
#call price
bs_call = bs_call_formula(X = 100, t = 0, K = 100, T = 0.25, rGrow = 0, r = 0, sigma = 0.294)
bs_call

5.859175592360259